In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [ ]:
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
module = fc.eightb.preselection.ranked_quadh
signal = ObjIter([Tree(fn, use_gen=False) for fn in module.signal_list[::3]])
bkg = ObjIter([Tree(module.QCD_B_List, use_gen=False), Tree(module.TTJets, use_gen=False)])

In [ ]:
study.quick(
    signal+bkg, legend=True,
    varlist=['n_loose_btag','n_medium_btag','n_tight_btag'],
    dim=(-1,3),
    efficiency=True
)

study.quick(
    signal+bkg, legend=True,
    varlist=['n_loose_btag','n_medium_btag','n_tight_btag'],
    dim=(-1,3),
    cumulative=-1, correlation=True, c_legend_frameon=True,
)

In [ ]:
(signal+bkg).apply(lambda t : build_collection(t, 'H\dY\d_b\d', 'jet', ptordered=True))
(signal+bkg).apply(lambda t : build_collection(t, 'H\dY\d', 'higgs', ptordered=True))

In [ ]:
def get_features(t):
  def get_j_features(t, i):
    label = {0:'even',1:'odd'}
    btagmulti = {f'{label.get(i)}_{wplabels[wp+1]}_btag':ak.sum(t.jet_btag[:,i::2]>jet_btagWP[wp+1], axis=-1) for wp in range(3)}
    btagmulti.update(
      **{f'{label.get(i)}_btagavg':ak.mean(t.jet_btag[:,i::2], axis=-1)}
    )
    return btagmulti


  t.extend(
    **get_j_features(t, 0),
    **get_j_features(t, 1),
  )

(signal+bkg).apply(get_features)

In [ ]:
study.quick(
    signal + bkg,
    varlist=[f'{y}_{var}' for y in ('even','odd') for var in ('loose_btag','medium_btag','tight_btag')],
    binlist=[np.arange(6)]*6,
    efficiency=True,
)

In [ ]:
%load helpers.py

In [ ]:
LL_z_r = grid_region_search('even_loose_btag', 'odd_loose_btag')

In [ ]:
is_sr = (RR[:,0,1] == 3) & (RR[:,1,1] == 3) & (RR[:,0,2] == 5) & (RR[:,1,2] == 5)
has_sr = (RR[:,0,2] > 3) & (RR[:,1,2] > 3)

eo_sr = sort_regions(RR, LL_z_r, is_sr)[:1]
eo_vr = sort_regions(RR, LL_z_r, ~has_sr)[:6]
regions = np.concatenate([eo_sr, eo_vr])
for x_r, y_r in regions:
    abcd_region = draw_abcd(x_r=x_r, y_r=y_r)
    study.quick2d(
    signal+bkg,
    xvarlist=['even_loose_btag'],
    yvarlist=['odd_loose_btag'],
    binlist=[np.arange(6)]*6,
    exe=abcd_region,
    # interp=True,
    )
    print(x_r, y_r)
    region_score(**abcd_region.store[0])
    plt.show()

In [ ]:
train_and_evaluate(
    features=['X_m','X_pt','X_eta','X_phi'],
    x_v='even_loose_btag', y_v='odd_loose_btag',
    x_r=eo_sr[0][0], y_r=eo_sr[0][1]
)

In [ ]:
train_and_evaluate(
    features=['X_m','X_pt','X_eta','X_phi'],
    x_v='even_loose_btag', y_v='odd_loose_btag',
    x_r=eo_vr[0][0], y_r=eo_vr[0][1]
)